In [1]:
#to read files from google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!git clone https://github.com/udacity/dermatologist-ai.git
!mkdir data

Cloning into 'dermatologist-ai'...
remote: Enumerating objects: 33, done.
remote: Total 33 (delta 0), reused 0 (delta 0), pack-reused 33
Unpacking objects: 100% (33/33), done.


In [3]:
!wget https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/train.zip
!wget https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/valid.zip
!wget https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/test.zip

--2019-04-20 09:20:26--  https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/train.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.116.80
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.116.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5736557430 (5.3G) [application/zip]
Saving to: ‘train.zip’

train.zip           100%[===================>]   5.34G  20.5MB/s    in 4m 28s  

2019-04-20 09:24:55 (20.4 MB/s) - ‘train.zip’ saved [5736557430/5736557430]

--2019-04-20 09:24:57--  https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/valid.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.120.40
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.120.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 864538487 (824M) [application/zip]
Saving to: ‘valid.zip’

valid.zip           100%[======

In [4]:
!unzip test.zip -d data
!unzip train.zip -d data
!unzip valid.zip -d data

Archive:  test.zip
   creating: data/test/
   creating: data/test/melanoma/
  inflating: data/test/melanoma/ISIC_0014559.jpg  
  inflating: data/test/melanoma/ISIC_0014772.jpg  
  inflating: data/test/melanoma/ISIC_0015206.jpg  
  inflating: data/test/melanoma/ISIC_0015167.jpg  
  inflating: data/test/melanoma/ISIC_0014434.jpg  
  inflating: data/test/melanoma/ISIC_0014077.jpg  
  inflating: data/test/melanoma/ISIC_0013073.jpg  
  inflating: data/test/melanoma/ISIC_0015133.jpg  
  inflating: data/test/melanoma/ISIC_0012395.jpg  
  inflating: data/test/melanoma/ISIC_0015163.jpg  
  inflating: data/test/melanoma/ISIC_0015193.jpg  
  inflating: data/test/melanoma/ISIC_0013072.jpg  
  inflating: data/test/melanoma/ISIC_0014703.jpg  
  inflating: data/test/melanoma/ISIC_0015180.jpg  
  inflating: data/test/melanoma/ISIC_0014883.jpg  
  inflating: data/test/melanoma/ISIC_0014912.jpg  
  inflating: data/test/melanoma/ISIC_0014336.jpg  
  inflating: data/test/melanoma/ISIC_0014800.jpg  
  infl

In [0]:
import torch
from torch import nn
from torch import optim
from torchvision import transforms, datasets, models
import numpy as np
from collections import OrderedDict

In [0]:
train_on_gpu = torch.cuda.is_available()

data_dir = 'data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

batch_size = 40

# TODO: Define your transforms for the training and validation sets
train_transforms = transforms.Compose([transforms.Resize(300),
                                transforms.RandomRotation(5),
                                transforms.RandomCrop(224),
                                transforms.RandomHorizontalFlip(0.1),
                                transforms.RandomVerticalFlip(0.1),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

valid_transforms = transforms.Compose([transforms.Resize(256),
                                transforms.RandomCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform=valid_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size)

['melanoma', 'nevus', 'seborrheic keratosis']


In [0]:
#loading a pre trained model, resnet152, and not freezing its layers
model = models.resnet152(pretrained=True)

classifier_name, old_classifier = model._modules.popitem()
#freezing all except the last
i = 0
for child in model.children():
  i += 1
  if i > 0:
    break
  for param in child.parameters():
    param.requires_grad = False

#add new hidden and dropout layers at the end
classifier_input_size = old_classifier.in_features
classifier = nn.Sequential(OrderedDict([('drop1', nn.Dropout(p=0.4)),
                                           ('fc1', nn.Linear(classifier_input_size, 3))
                                           ]))
model.add_module(classifier_name, classifier)
#model

In [0]:
#defining hyper-parameters
epochs = 35
learning_rate = 0.001

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), learning_rate, momentum=0.91)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)

In [0]:
#training the model
if train_on_gpu:
    model.cuda()

valid_loss_min = np.Inf

for epoch in range(epochs):
    
    model.train()
    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss=0.0
    accuracy = 0.0
    running_corrects = 0.0
    dataset_size = 0
    
    for images, labels in trainloader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            images, labels = images.cuda(), labels.cuda()
            
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(images)

        # calculate the batch loss
        loss = criterion(output, labels)
        
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        
        # perform a single optimization step (parameter update)
        optimizer.step()
        
        # update training loss
        train_loss += loss.item()
        
    #validation starts here
    model.eval()
    with torch.no_grad():
        for images, labels in validloader:
            if train_on_gpu:
                images, labels = images.cuda(), labels.cuda()
            output = model(images)
            loss = criterion(output, labels)
            valid_loss += loss.item()
            
            _, preds = torch.max(output, 1)
            running_corrects += torch.sum(preds == labels.data)
            dataset_size += len(images)
        
    lr_scheduler.step(valid_loss_min)
    # Calculate accuracy
    accuracy = running_corrects.double() / dataset_size
    
    print("Epoch #", epoch, ':\t', "Training loss: ({:.3f})".format(train_loss/len(trainloader)), '\t',
          "Validation loss: ({:.3f})".format(valid_loss/len(validloader)), '\t',
          "Validation accuracy: ({:.3f})".format(accuracy))
    
    # save model if validation loss has decreased
    if valid_loss/len(validloader) <= valid_loss_min:
        print('Validation loss decreased ({:.3f} --> {:.3f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss/len(validloader)))
        torch.save(model.state_dict(), 'model_dermatologist_ai.pt')
        valid_loss_min = valid_loss/len(validloader)

Epoch # 0 :	 Training loss: (0.838) 	 Validation loss: (0.832) 	 Validation accuracy: (0.633)
Validation loss decreased (inf --> 0.832).  Saving model ...
Epoch # 1 :	 Training loss: (0.649) 	 Validation loss: (0.811) 	 Validation accuracy: (0.647)
Validation loss decreased (0.832 --> 0.811).  Saving model ...
Epoch # 2 :	 Training loss: (0.553) 	 Validation loss: (0.640) 	 Validation accuracy: (0.740)
Validation loss decreased (0.811 --> 0.640).  Saving model ...
Epoch # 3 :	 Training loss: (0.469) 	 Validation loss: (0.653) 	 Validation accuracy: (0.727)
Epoch # 4 :	 Training loss: (0.425) 	 Validation loss: (0.742) 	 Validation accuracy: (0.713)
Epoch # 5 :	 Training loss: (0.366) 	 Validation loss: (0.616) 	 Validation accuracy: (0.753)
Validation loss decreased (0.640 --> 0.616).  Saving model ...
Epoch # 6 :	 Training loss: (0.317) 	 Validation loss: (0.597) 	 Validation accuracy: (0.807)
Validation loss decreased (0.616 --> 0.597).  Saving model ...
Epoch # 7 :	 Training loss: (

In [0]:
#authemticate pydrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#upload the checkpoint to the drive
upload = drive.CreateFile({'title': 'model_dermatologist_ai_84.pt'})
upload.SetContentFile('model_dermatologist_ai.pt')
upload.Upload()

#laoding the best most accurate checkpoint
state_dict = torch.load('model_dermatologist_ai.pt')
model.load_state_dict(state_dict)

In [0]:
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

test_data = ImageFolderWithPaths(test_dir, transform=valid_transforms)
testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

In [0]:
import pandas as pd
#classes = [name.replace('_', ' ') for name in test_data.classes]
#print(classes)

#creating dictionary and lists for the csv file
csv_dict = {}
id_values = []
task_1_values = []
task_2_values = []

#testing the model
model.eval()
test_loss = 0.0
dataset_size = 0
running_corrects = 0

with torch.no_grad():
  for images, labels, paths in testloader:
    if train_on_gpu:
        images, labels = images.cuda(), labels.cuda()
    output = model(images)
    loss = criterion(output, labels)
    test_loss += loss.item()

    _, preds = torch.max(output, 1)
    running_corrects += torch.sum(preds == labels.data)
    dataset_size += len(images)
    
    output = nn.functional.softmax(output)
    #print(output[0:5])
    
    #adding values to the columns
    id_values.extend(paths)
    #print(id_values)
    
    for each_output in output:
      task_1_values.append(each_output.tolist()[0])
      task_2_values.append(each_output.tolist()[2])
      
    #print(task_1_values)
    #print(task_2_values)

# Calculate accuracy
accuracy = running_corrects.double() / dataset_size

print("Test loss: ({:.3f})".format(test_loss/len(testloader)), '\t',
      "Test accuracy: ({:.3f})".format(accuracy))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Test loss: (0.692) 	 Test accuracy: (0.755)


In [0]:
#exporting predictions to a csv file
csv_dict['Id'] = id_values
csv_dict['task_1'] = task_1_values
csv_dict['task_2'] = task_2_values

df = pd.DataFrame.from_dict(csv_dict, orient = 'columns')
df.to_csv('predictions_84.csv', index = False)

In [0]:
#upoading the csv file to gdrive
!ls
upload = drive.CreateFile({'title': 'predictions_84.csv'})
upload.SetContentFile('predictions_84.csv')
upload.Upload()

adc.json  dermatologist-ai  model_dermatologist_ai.pt  sample_data  train.zip
data	  drive		    predictions_84.csv	       test.zip     valid.zip


In [0]:
#installing requirements to run evaluations
!pip install -r dermatologist-ai/requirements.txt

/bin/bash: conda: command not found
/bin/bash: activate: No such file or directory
    100% |████████████████████████████████| 14.7MB 1.8MB/s 
    100% |████████████████████████████████| 18.9MB 1.7MB/s 
    100% |████████████████████████████████| 11.8MB 3.9MB/s 
    100% |████████████████████████████████| 50.0MB 810kB/s 
yellowbrick 0.9.1 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.18.1 which is incompatible.
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
imbalanced-learn 0.4.3 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.18.1 which is incompatible.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.19.2 which is incompatible.
fbprophet 0.4.post2 has requirement pandas>=0.20.1, but you'll have pandas 0.19.2 which is incompatible.
fastai 1.0.46 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
fancyimpute 0.4.2 has requirement scikit-learn

In [0]:
#evaluating the predictions
!python ./get_results.py /content/predictions_84.csv 0.35

Category 1 Score: 0.865
Category 2 Score: 0.914
Category 3 Score: 0.890
